![ALT_TEXT_FOR_SCREEN_READERS](./header.png)

# Exercise 4.A LLM Basics and Tools using Colab and Mistral LLM

The goal of this exercise is to setup a connection from the notebook to a local or remote large language model (LLM). Using this connection
the basic modes of working with LLMs shall be tested:
- Text Completion: complete a text. A query text is given and the model continues this text.
- Chat: A list of system and user messages is given and the model generates the next message.

We are using **Mistral.ai**[1] for cloud execution of the LLM and the framework **langchain**[2] for the access to the model.

- [1] [https://mistral.ai//](https://mistral.ai/)
- [2] [https://www.langchain.com/](https://www.langchain.com/)


<a target="_blank" href="https://colab.research.google.com/github/ditomax/mlpython/blob/main/Exercise%204.A%20LLM%20Basics%20and%20Tools%20Colab%20Mistral.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



# Considerations

- Read the tutorial carefully
- Retrieve a Mistral API Key [1] 
- Install **langchain_mistralai** and **langchain** packages into your environment (use pip inside the workbook and comment it out later)

# Requirements

- R0: Instantiate a text completion model object and improve the head of the story for completion (10%)
- R1: Instantiate a chat completion model object and work to improve the system prompt message (10%)
- R2: Test the effect of the temperature parameter values 0.0 and 1.0 in the chat completion example (20%)
- R3: Extend the structured output of the joke class (20%)
- R4: Check the produced tool message for correct parameters (10%)
- R5: Extend the tool calling agent by a system date and time tool (30%)

# Setup

In [ ]:
%pip install -U langchain_mistralai

In [ ]:
%pip install -U langchain

# Text Completion

Text completion means that the LLM takes the given text as start of a story and continues to generate tokens which extend the story.
Such a model interface takes a string as input and generates a string as output.

In [ ]:
import os
from langchain_mistralai import ChatMistralAI

In [ ]:
os.environ["MISTRAL_API_KEY"] = '...'

In [ ]:
llm = ChatMistralAI(model="mistral-small-2506",temperature=0,max_retries=1,)

In [ ]:
#
# R0: improve the start of the story to resemble your favorite fairy tale...
#
result = llm.invoke('Once upon a time, in a dark tyrolian valley ')
print(result)

# Chat Completion

A chat completion model takes a list of messages as input and generates the next message. The resulting message is of type **AIMessage**.
The list of messages can contain tuples with role and prompt for different message objects from the langchain_core.messages module.

**Note:** The chat model is a different interface compared to the completion model.

In [ ]:
from langchain_core.messages import AIMessage


In [ ]:
llm = ChatMistralAI(model="mistral-small-2506",temperature=0,max_retries=1,)

In [ ]:
#
# R1: improve the system prompt such that it contains all elements as learned in the slides.
#

messages = [
    ("system", "You are a skilled philospher. You can explain complext information in simple terms." ),
    ("human",  "What is the meaning of life?"),
]

In [ ]:
# call the LLM with the message
ai_msg = llm.invoke(messages)
print(ai_msg)

In [ ]:
# specific return value
print(ai_msg.content)

In [ ]:
# other metadata
print(ai_msg.response_metadata)

In [ ]:
#
# R2: test the impact of the temperature parameter (1.0) using the same prompt.
#

# Structured Output

In [ ]:
from pydantic import BaseModel, Field

In [ ]:
class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    explanation: str = Field(description="Detailed explanation why the joke if funny.")

In [ ]:
llm = ChatMistralAI(model="mistral-small-2506",temperature=0,max_retries=1,)

In [ ]:
structured_llm = llm.with_structured_output(Joke)

In [ ]:
ai_msg = structured_llm.invoke("Tell me a joke about AIs.")

In [ ]:
print(ai_msg)

In [ ]:
#
# R3: extend the structured output by an explanation field which contains the reasoning, why this is actually funny.
#

# Tool use

Tool use is a way to extend the function of LLMs. This is done in the following steps:
1. in the first step, the tools are defined and the definitions are passed to the LLM as context information. The LLM is instructed to extract optional tool use, including tool name and tool parameters, if the user query requests a tool use. In this case the LLM generates a tool use message with all required parameters.
1. in the second step, a piece of application logic takes the tool call messages and executes the tool calls. The results are placed back in the message list.
1. in the last step, the tool results are used to formulation the answer.

Those steps are usually stitched together in an agent.

In [ ]:
from langchain_core.tools import tool
import math

In [ ]:
#
# @tool is an annotator which adds specific properties to your tool function. Those properties allow the LLM to understand the parameters and the output of the tool.
#
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int



In [ ]:
@tool
def square_root(x: float) -> float:
    """returns the square root of x."""
    return math.sqrt(x)



In [ ]:
# call tool for testing
print(multiply.invoke({"first_int": 4, "second_int": 5}))

In [ ]:
# call tool with wrong parameters for testing
try:
    print(multiply.invoke({"first_int": 4.1, "second_int": 5}))
except Exception as e:
    print(e)

In [ ]:
print(square_root.invoke({'x':25.0}))

In [ ]:
#
# prepare LLM for tool use
#
tools = [multiply,square_root]

In [ ]:
llm = ChatMistralAI(model="mistral-small-2506",temperature=0,max_retries=1,)

In [ ]:
#
# bind tools to LLM
#
llm_with_tools = llm.bind_tools(tools)

In [ ]:
query = 'what is ten times 7ish? Use tool calling.'

In [ ]:
#
# R4: check the resulting tool message for the correct tool parameters. Extract the tool calling values (tool name, tool parameters). Those parameter can be used to call the python tool.
#
result = llm_with_tools.invoke(query)

In [ ]:
result

In [ ]:
result.tool_calls

# Tool calling agent

The langchain tool calling agent is a class which integrates all three steps of tool calling into one object.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful calculation assistant. Use your tools where it is appropriate. Mimic the style of the user query."), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

In [ ]:
llm = ChatMistralAI(model="mistral-small-2506",temperature=0,max_retries=1,)

In [ ]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
print(agent_executor.invoke({"input": "what is the square root of ten times 7ish?", }))

# Agent with system time

In [ ]:
#
# R5: extend the tool calling agent by a tool which returns the current date and time as string. The LLMs usually do now know date and time!
#